In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import bibtexparser


# Proof of Concept

In [33]:
# BASE SEARCH STRING
search_test = "https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&offset=100&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance"

#GET PAGE
r = requests.get(search_test, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0'})

#GET PAPER IDs
soup = BeautifulSoup(r.content, 'html5lib') 
s_id = soup.find_all("a", {"class": "result-list-title-link u-font-serif text-s"})
piis = [x.get("href").split('/')[-1] for x in s_id]

# GET BIBTEX FROM API
rAPI = requests.post('https://www.sciencedirect.com/search/api/export-citations', proxies={'https':"https://127.0.0.1:8080"}, data={'type':'bibtex', 'pii':piis}, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0'},verify=False)

# PARSE
bibdata = bibtexparser.loads(rAPI.content.decode())

dic = {'title':[], 'journal':[], 'year':[], 'keywords':[]}

for bib_id in bibdata.entries:
    try:
        dic['title'].append(bib_id["title"])
    except(KeyError):
        dic['title'].append("")
        continue
    try:
        dic['journal'].append(bib_id['journal'])
    except(KeyError):
        dic['joural'].append("")
        continue
    try:
        dic['year'].append(bib_id['year'])
    except(KeyError):
        dic['year'].append("")
        continue    
    try:
        dic['keywords'].append(bib_id['keywords'])
    except(KeyError):
        dic['keywords'].append("")
        continue

#INTO DATA FRAME
df = pd.DataFrame.from_dict(dic)
df.head()

/home/caio/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,title,journal,year,keywords
0,Cyclist-aware traffic lights through distribut...,Pervasive and Mobile Computing,2016,"Smartphone sensing, Distributed model, Power c..."
1,Towards gadget-free internet services: A roadm...,Telematics and Informatics,2018,
2,Querytogether: Enabling entity-centric explora...,Information Processing & Management,2018,"Collaborative search, Exploratory search, Co-l..."
3,DynMAC: A resistant MAC protocol to coexistenc...,Computer Networks,2015,"Wireless sensor network, Dynamic channel recon..."
4,Design of a secure smart card-based multi-serv...,Journal of Information Security and Applications,2016,"Multi-server environment, Authentication, Smar..."


# Extracting CSV from Elsevier search (TOP 1000)

In [37]:
offset=0 # OFFSET page, iterate summing +100
searchURL = "https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance"
paperIds = []

dic = {'title':[], 'journal':[], 'year':[], 'keywords':[]}

while offset < 1000:
    searchOffset = searchURL + "&offset=" + str(offset)
    print(searchOffset)
    #GET PAGE
    r = requests.get(searchOffset, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0'})
   
    #GET PAPER IDs
    soup = BeautifulSoup(r.content, 'html5lib') 
    s_id = soup.find_all("a", {"class": "result-list-title-link u-font-serif text-s"})
    piis = [x.get("href").split('/')[-1] for x in s_id]

    # GET BIBTEX FROM API
    rAPI = requests.post('https://www.sciencedirect.com/search/api/export-citations', data={'type':'bibtex', 'pii':piis}, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0'})
    
    # BIBTEXT PARSE
    bibdata = bibtexparser.loads(rAPI.content.decode())
    
    # PUT INFORMATION IN DICTIONARY
    for bib_id in bibdata.entries:
        try:
            dic['title'].append(bib_id["title"])
        except(KeyError):
            dic['title'].append("")
            continue
        try:
            dic['journal'].append(bib_id['journal'])
        except(KeyError):
            dic['joural'].append("")
            continue
        try:
            dic['year'].append(bib_id['year'])
        except(KeyError):
            dic['year'].append("")
            continue    
        try:
            dic['keywords'].append(bib_id['keywords'])
        except(KeyError):
            dic['keywords'].append("")
            continue
    
    offset = offset + 100

#DICTIONARY INTO DATAFRAME
df = pd.DataFrame.from_dict(dic)
df.to_csv("elsevier.csv",index=False)
df.head()

https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance&offset=0
https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance&offset=100
https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance&offset=200
https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance&offset=300
https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance&offset=400
https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&sortBy=relevance&offset=500
https://www.sciencedirect.com/search/advanced?articleTypes=FLA&date=2015-2020&qs=%22Ubiquitous%20Computing%22&show=100&s

,title,journal,year,keywords
0,A context-aware approach in realization of soc...,Robotics and Computer-Integrated Manufacturing,2016,"Cognitive robotics, Context-awareness, Probabi..."
1,Efficient Management of Security for Supportin...,Procedia Computer Science,2019,"Ubiquitous system, Intrusion detection system,..."
2,Visualizing the knowledge profile on self-powe...,Nano Energy,2018,"Self-powered technology, Knowledge profile, Na..."
3,An Object-Oriented Model for Object Orchestrat...,Procedia Computer Science,2017,"Smart object, ubiquitous computing, modeling, ..."
4,Privacy protection by typing in ubiquitous com...,Journal of Systems and Software,2016,"Privacy, Type system, Ubiquitous computing, Pe..."
